In [133]:
import pandas as pd
from unidecode import unidecode

pd.options.display.float_format = "{:.2f}".format

In [134]:
secu_alim = pd.read_csv('../database/securite_alimentaire_2020.csv', dtype=str)
def preprocess_securite_alimentaire_data(securite_alimentaire):
    securite_alimentaire = securite_alimentaire.drop(columns=['Code année', 'Année', 'Unité', 'Code Élément', 'Élément', 'Code Produit', 'Domaine', 'Code Domaine', 'Note', 'Description du Symbole', 'Symbole'])
    return securite_alimentaire
securite_alimentaire = preprocess_securite_alimentaire_data(secu_alim)
securite_alimentaire

,Code zone (FAO),Zone,Produit,Valeur
0,2,Afghanistan,Suffisance des apports énergétiques alimentair...,105
1,2,Afghanistan,Disponibilité alimentaire par habitant utilisé...,2266
2,2,Afghanistan,Disponibilité alimentaire par habitant utilisé...,2245
3,2,Afghanistan,"Part des céréales, des racines et des tubercul...",70
4,2,Afghanistan,Disponibilités protéiques moyennes (g/personne...,61.9
...,...,...,...,...
13883,181,Zimbabwe,Besoins énergétiques alimentaires minimaux (kc...,1743
13884,181,Zimbabwe,Besoins énergétiques alimentaires moyens (kcal...,2245
13885,181,Zimbabwe,Coefficient de variation de la distribution de...,0.31
13886,181,Zimbabwe,Incidence des pertes caloriques au niveau de l...,2.73


In [135]:
def convert_secu_alim(securite_alimentaire):
    securite_alimentaire['Valeur'] = pd.to_numeric(securite_alimentaire['Valeur'], errors='coerce')
    return securite_alimentaire

securite_alimentaire_transformed = convert_secu_alim(securite_alimentaire)
securite_alimentaire_transformed

,Code zone (FAO),Zone,Produit,Valeur
0,2,Afghanistan,Suffisance des apports énergétiques alimentair...,105.00
1,2,Afghanistan,Disponibilité alimentaire par habitant utilisé...,2266.00
2,2,Afghanistan,Disponibilité alimentaire par habitant utilisé...,2245.00
3,2,Afghanistan,"Part des céréales, des racines et des tubercul...",70.00
4,2,Afghanistan,Disponibilités protéiques moyennes (g/personne...,61.90
...,...,...,...,...
13883,181,Zimbabwe,Besoins énergétiques alimentaires minimaux (kc...,1743.00
13884,181,Zimbabwe,Besoins énergétiques alimentaires moyens (kcal...,2245.00
13885,181,Zimbabwe,Coefficient de variation de la distribution de...,0.31
13886,181,Zimbabwe,Incidence des pertes caloriques au niveau de l...,2.73


In [136]:
def transform_secu_alim(securite_alimentaire):
    grouper_lignes = securite_alimentaire.groupby(['Code zone (FAO)', 'Zone', 'Produit'], as_index=False).sum()
    return grouper_lignes
grouper_lignes_transformed = transform_secu_alim(securite_alimentaire)
grouper_lignes_transformed  

,Code zone (FAO),Zone,Produit,Valeur
0,1,Arménie,Besoins énergétiques alimentaires minimaux (kc...,1872.00
1,1,Arménie,Besoins énergétiques alimentaires moyens (kcal...,2420.00
2,1,Arménie,Coefficient de variation de la distribution de...,0.21
3,1,Arménie,Densité du réseau ferroviaire (pour 100 km car...,2.30
4,1,Arménie,Disponibilité alimentaire par habitant utilisé...,3218.00
...,...,...,...,...
8783,99,Islande,Stabilité politique et absence de violence/ter...,1.37
8784,99,Islande,Suffisance des apports énergétiques alimentair...,142.00
8785,99,Islande,Taux de dépendance à l'égard des importations ...,92.50
8786,99,Islande,Valeur des importations alimentaires par rappo...,10.00


In [137]:
def transform_secu_alim(grouper_lignes_transformed):
    securite_alimentaire = grouper_lignes_transformed.pivot(index=['Code zone (FAO)','Zone'], columns='Produit', values='Valeur')
    securite_alimentaire = securite_alimentaire.reset_index()
    securite_alimentaire = securite_alimentaire.rename(columns={'Code zone (FAO)': 'country_code'})
    securite_alimentaire = securite_alimentaire.fillna(0)
    num_cols = securite_alimentaire.select_dtypes(include='number').columns
    securite_alimentaire[num_cols] = securite_alimentaire[num_cols].round(2)
    return securite_alimentaire

securite_alimentaire = transform_secu_alim(grouper_lignes_transformed)
securite_alimentaire

Produit,country_code,Zone,Besoins énergétiques alimentaires minimaux (kcal/personne/jour),Besoins énergétiques alimentaires moyens (kcal/personne/jour),Coefficient de variation de la distribution de la consommation calorique habituelle (vraie valeur),Densité du réseau ferroviaire (pour 100 km carrés de surface totale du pays),Disponibilité alimentaire par habitant utilisée dans l'estimation de la prévalence de la sous-alimentation (kcal/personne/jour),Disponibilité alimentaire par habitant utilisée dans l'estimation de la prévalence de la sous-alimentation (kcal/personne/jour) (moyenne sur 3 ans),Disponibilités moyennes de matière grasse (g/personne/jour) (moyenne sur 3 ans),Disponibilités protéines moyennes d’origine animale (g/personne/jour) (moyenne sur 3 ans),...,Prévalence de l’insécurité alimentaire grave – hommes adultes (%) (moyenne sur 3 ans),Prévalence de l’insécurité alimentaire grave – population totale (%) (moyenne sur 3 ans),Prévalence de l’insécurité alimentaire modérée ou grave – femmes adultes (%) (moyenne sur 3 ans),Prévalence de l’insécurité alimentaire modérée ou grave – hommes adultes (%) (moyenne sur 3 ans),Prévalence de l’insécurité alimentaire modérée ou grave – population totale (%) (moyenne sur 3 ans),Stabilité politique et absence de violence/terrorisme (indice),Suffisance des apports énergétiques alimentaires moyens (%) (moyenne sur 3 ans),Taux de dépendance à l'égard des importations céréalières (%) (moyenne sur 3 ans),Valeur des importations alimentaires par rapport aux exportations totales de marchandises (%) (moyenne sur 3 ans),Variabilité des disponibilités alimentaires par habitant (Kcal/personne/jour)
0,1,Arménie,1872.00,2420.00,0.21,2.30,3218.00,3226.00,113.10,55.00,...,0.00,0.00,27.30,22.50,24.90,-0.77,133.00,85.90,24.00,51.00
1,10,Australie,1920.00,2493.00,0.21,0.00,3480.00,3479.00,158.00,81.20,...,12.60,10.90,36.00,35.70,35.80,0.86,140.00,-163.20,5.00,31.00
2,100,Inde,1797.00,2325.00,0.27,2.10,2552.00,2551.00,60.40,17.70,...,0.00,0.00,0.00,0.00,0.00,-0.84,110.00,-8.90,6.00,20.00
3,101,Indonésie,1791.00,2311.00,0.28,0.00,2911.00,2906.00,75.40,28.20,...,0.00,0.00,15.30,15.30,15.30,-0.46,126.00,15.30,8.00,43.00
4,102,Iran (République islamique d'),1841.00,2376.00,0.27,0.50,2919.00,2907.00,78.50,26.80,...,24.20,23.10,124.50,130.00,127.20,-1.70,122.00,47.80,14.00,37.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,95,Honduras,1781.00,2304.00,0.32,0.00,2618.00,2639.00,87.50,24.80,...,50.70,53.60,160.50,138.80,149.60,-0.45,115.00,65.30,18.00,8.00
200,96,Chine - RAS de Hong-Kong,1880.00,2432.00,0.22,0.00,3099.00,3113.00,128.10,103.50,...,0.00,0.00,0.00,0.00,0.00,0.06,128.00,100.00,4.00,73.00
201,97,Hongrie,1945.00,2527.00,0.21,8.20,3368.00,3366.00,150.30,56.00,...,4.70,6.20,35.70,27.80,31.80,0.84,133.00,-91.80,4.00,26.00
202,98,Croatie,1920.00,2492.00,0.21,3.00,3219.00,3220.00,124.80,66.20,...,3.40,4.90,41.70,26.70,34.20,0.61,129.00,-51.00,17.00,34.00


In [138]:
def transform_secu_alim(securite_alimentaire):
    securite_alimentaire.columns = securite_alimentaire.columns.str.replace(' ', '_', regex=False)
    securite_alimentaire.columns = securite_alimentaire.columns.str.lower()
    securite_alimentaire.columns = [unidecode(col) for col in securite_alimentaire.columns]
    return securite_alimentaire

securite_alimentaire = transform_secu_alim(securite_alimentaire)
securite_alimentaire

,country_code,zone,besoins_energetiques_alimentaires_minimaux_(kcal/personne/jour),besoins_energetiques_alimentaires_moyens_(kcal/personne/jour),coefficient_de_variation_de_la_distribution_de_la_consommation_calorique_habituelle_(vraie_valeur),densite_du_reseau_ferroviaire_(pour_100_km_carres_de_surface_totale_du_pays),disponibilite_alimentaire_par_habitant_utilisee_dans_l'estimation_de_la_prevalence_de_la_sous-alimentation_(kcal/personne/jour),disponibilite_alimentaire_par_habitant_utilisee_dans_l'estimation_de_la_prevalence_de_la_sous-alimentation_(kcal/personne/jour)_(moyenne_sur_3_ans),disponibilites_moyennes_de_matiere_grasse_(g/personne/jour)_(moyenne_sur_3_ans),disponibilites_proteines_moyennes_d'origine_animale_(g/personne/jour)_(moyenne_sur_3_ans),...,prevalence_de_l'insecurite_alimentaire_grave_-_hommes_adultes_(%)_(moyenne_sur_3_ans),prevalence_de_l'insecurite_alimentaire_grave_-_population_totale_(%)_(moyenne_sur_3_ans),prevalence_de_l'insecurite_alimentaire_moderee_ou_grave_-_femmes_adultes_(%)_(moyenne_sur_3_ans),prevalence_de_l'insecurite_alimentaire_moderee_ou_grave_-_hommes_adultes_(%)_(moyenne_sur_3_ans),prevalence_de_l'insecurite_alimentaire_moderee_ou_grave_-_population_totale_(%)_(moyenne_sur_3_ans),stabilite_politique_et_absence_de_violence/terrorisme_(indice),suffisance_des_apports_energetiques_alimentaires_moyens_(%)_(moyenne_sur_3_ans),taux_de_dependance_a_l'egard_des_importations_cerealieres_(%)_(moyenne_sur_3_ans),valeur_des_importations_alimentaires_par_rapport_aux_exportations_totales_de_marchandises_(%)_(moyenne_sur_3_ans),variabilite_des_disponibilites_alimentaires_par_habitant_(kcal/personne/jour)
0,1,Arménie,1872.00,2420.00,0.21,2.30,3218.00,3226.00,113.10,55.00,...,0.00,0.00,27.30,22.50,24.90,-0.77,133.00,85.90,24.00,51.00
1,10,Australie,1920.00,2493.00,0.21,0.00,3480.00,3479.00,158.00,81.20,...,12.60,10.90,36.00,35.70,35.80,0.86,140.00,-163.20,5.00,31.00
2,100,Inde,1797.00,2325.00,0.27,2.10,2552.00,2551.00,60.40,17.70,...,0.00,0.00,0.00,0.00,0.00,-0.84,110.00,-8.90,6.00,20.00
3,101,Indonésie,1791.00,2311.00,0.28,0.00,2911.00,2906.00,75.40,28.20,...,0.00,0.00,15.30,15.30,15.30,-0.46,126.00,15.30,8.00,43.00
4,102,Iran (République islamique d'),1841.00,2376.00,0.27,0.50,2919.00,2907.00,78.50,26.80,...,24.20,23.10,124.50,130.00,127.20,-1.70,122.00,47.80,14.00,37.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,95,Honduras,1781.00,2304.00,0.32,0.00,2618.00,2639.00,87.50,24.80,...,50.70,53.60,160.50,138.80,149.60,-0.45,115.00,65.30,18.00,8.00
200,96,Chine - RAS de Hong-Kong,1880.00,2432.00,0.22,0.00,3099.00,3113.00,128.10,103.50,...,0.00,0.00,0.00,0.00,0.00,0.06,128.00,100.00,4.00,73.00
201,97,Hongrie,1945.00,2527.00,0.21,8.20,3368.00,3366.00,150.30,56.00,...,4.70,6.20,35.70,27.80,31.80,0.84,133.00,-91.80,4.00,26.00
202,98,Croatie,1920.00,2492.00,0.21,3.00,3219.00,3220.00,124.80,66.20,...,3.40,4.90,41.70,26.70,34.20,0.61,129.00,-51.00,17.00,34.00


In [139]:
securite_alimentaire.columns

Index(['country_code', 'zone',
       'besoins_energetiques_alimentaires_minimaux_(kcal/personne/jour)',
       'besoins_energetiques_alimentaires_moyens_(kcal/personne/jour)',
       'coefficient_de_variation_de_la_distribution_de_la_consommation_calorique_habituelle_(vraie_valeur)',
       'densite_du_reseau_ferroviaire_(pour_100_km_carres_de_surface_totale_du_pays)',
       'disponibilite_alimentaire_par_habitant_utilisee_dans_l'estimation_de_la_prevalence_de_la_sous-alimentation_(kcal/personne/jour)',
       'disponibilite_alimentaire_par_habitant_utilisee_dans_l'estimation_de_la_prevalence_de_la_sous-alimentation_(kcal/personne/jour)_(moyenne_sur_3_ans)',
       'disponibilites_moyennes_de_matiere_grasse_(g/personne/jour)_(moyenne_sur_3_ans)',
       'disponibilites_proteines_moyennes_d'origine_animale_(g/personne/jour)_(moyenne_sur_3_ans)',
       'disponibilites_proteiques_moyennes_(g/personne/jour)_(moyenne_sur_3_ans)',
       'incidence_des_pertes_caloriques_au_niveau_de_la_ve

In [140]:
columns_millions = ["nombre_d'adultes_(18_ans_ou_plus)_obeses_(millions)",
       "nombre_d'enfants_de_moins_de_5_ans_presentant_un_exces_ponderal_(estimation_modelisee)_(millions)",
       "nombre_d'enfants_de_moins_de_5_ans_presentant_un_retard_de_croissance_(estimation_modelisee)_(millions)",
       "nombre_d'enfants_de_moins_de_5_ans_emacies_(millions)",
       "nombre_de_femmes_adultes_en_situation_d'insecurite_alimentaire_grave_(millions)_(moyenne_sur_3_ans)",
       "nombre_de_femmes_adultes_en_situation_d'insecurite_alimentaire_moderee_ou_grave_(millions)_(moyenne_sur_3_ans)",
       "nombre_de_nouveaux-nes_en_insuffisance_ponderale_(millions)",
       "nombre_de_personnes_en_situation_d'insecurite_alimentaire_grave_(millions)_(moyenne_sur_3_ans)",
       "nombre_de_personnes_en_situation_d'insecurite_alimentaire_moderee_ou_grave_(millions)_(moyenne_sur_3_ans)",
       "nombre_de_personnes_sous-alimentees_(millions)_(moyenne_sur_3_ans)",
       "nombre_d'hommes_adultes_en_situation_d'insecurite_alimentaire_moderee_ou_grave_(millions)_(moyenne_sur_3_ans)",
       "pib_par_habitant,_($_ppa_internationaux_constants_de_2017)"]

def convert_millions_to_units(securite_alimentaire):
       securite_alimentaire[columns_millions] = securite_alimentaire[columns_millions] * 1000000
       return securite_alimentaire

securite_alimentaire = convert_millions_to_units(securite_alimentaire)


In [141]:
def save_transformed_data(securite_alimentaire):
    securite_alimentaire.to_csv('../export/securite_alimentaire_2020.csv', index=False, encoding='utf-8')
    print("Transformed data saved successfully to 'securite_alimentaire_2020.csv'!")

save_transformed_data(securite_alimentaire)

Transformed data saved successfully to 'securite_alimentaire_2020.csv'!
